In [ ]:
import matplotlib
#%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser

In [ ]:
import os

In [ ]:
hparams = create_hparams()
hparams.sampling_rate = 22050
checkpoint_path = "checkpoint_15000"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path, map_location = 'cpu')['state_dict'])  #added map_location = 'cpu'
_ = model.eval()          #it was originally model.cuda().eval().half()
waveglow_path = 'waveglow_256channels.pt'
waveglow = torch.load(waveglow_path, map_location = 'cpu')['model']   #added map_location = 'cpu'
waveglow.eval()   #originally waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
#denoiser = Denoiser(waveglow)


In [ ]:
import soundfile as sf

In [ ]:
import time

In [ ]:
def convert(array):
    sf.write('sample.wav', array, 22050)
    os.system('ffmpeg -i {0} -filter:a "atempo=0.80" {1}'.format('sample.wav', 'sample0.wav'))
    #os.system('ffmpeg -i {0} -ar 8000 {1}'.format('sample0.wav', 'sample1.wav'))
    data, rate = sf.read('sample0.wav')
    os.remove('sample.wav')
    os.remove('sample0.wav')
    #os.remove('sample1.wav')
    return data

In [ ]:
def speech(t):
    start = time.time()
    text = t
    sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
    print(sequence)
    sequence = torch.autograd.Variable(
        torch.from_numpy(sequence)).long()   #originally torch.from_numpy(sequence)).cuda().long()
    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    with torch.no_grad():
        audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
    #audio_denoised = denoiser(audio, strength=0.01)[:, 0]
    data = convert(audio[0].data.cpu().numpy())
    #os.system('ffmpeg -i {0} -filter:a "atempo=0.85" {1}'.format('harvard_inference/audio/'+str(i)+'.wav', 'harvard_inference/audio_0.85/'+str(i)+'.wav'))
    aud = ipd.Audio(data, rate=22050)
    end = time.time()
    print(end-start)
    return aud

In [ ]:
speech('I understand your frustration and disappointment. I am sorry that its happening and I would like to help prevent it in the future. What style of diapers did you buy? For instance, was it the snugglers, pull ups or baby dry.')

In [5]:
from final import display,speech,play_device
import pyaudio

In [13]:
data = speech('Thank you for calling Huggies. How may I help you today .')

15.046638011932373


In [14]:
display(data)

In [16]:
def play_device(data):
    audio_interface = pyaudio.PyAudio()
    _audio_stream = audio_interface.open(format=pyaudio.paInt16,channels=1, rate=16000,output=True)
    _audio_stream.write(data.tostring())
#     _audio_stream.close()
play_device(data)